<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Enterprise Vector Store - Embedding and Search in SQL
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Overview</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage provides a suite of new in-database analytic capabilities for Vector storage, Management, Indexing, and Search, including</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Vector Datatype</b> based VARBYTE arrays</li>
    <li><b>Normalization</b> functions to improve search efficiencies</li>
    <li><b>Vector Indexing and Search</b> leveraging multiple algorithms</li>
    </ul>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Vector Datatype using TD_AITextEmbeddings</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Currently, the Vantage Database supports several different methods for generating <b>Vector Embeddings</b> including in-database Bring Your Own Model (BYOM) functions, in-platform GPU-accelerated open-source model inferencing, and API-supported embedding using Cloud-based Large Language Models. </p>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration uses some of the Consumer Financial Protection Board complaints data to illustrate a SQL-based end-to-end Vector Embedding use case:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Generate <b>Vector Embeddings</b> using Cloud-based LLMs</li>
    <li>Normalize the vector data for efficient search</li>
    <li>Calculate Vector Distance between complaints and topics data</li>
    <li>Create an HSNW index and use it for highly-scalable distance analysis</li>
    </ol>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a new named connection if necessary using the %addconnect magic</p> 

In [1]:
%addconnect name=vs_demo, host=34.232.150.25

Success: 'vs_demo' connection added

In [44]:
%disconnect vs_demo

Success: 'vs_demo' disconnected

In [74]:
%connect vs_demo, user=data_engineer

Connected: 'vs_demo' connection activated for user 'data_engineer'


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 1 - Generate Vector Embeddings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function TD_AITextEmbeddings can use either built-in or user-defined CSP LLM endpoints and models for generating vector embedding.  The built-in capabilities follow the model support matrix provded in the User Guide.</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>View the original dataset.  CFPB Consumer Complaints</li>
    <li>Set up authorization.  To use built-in LLM services, keep the USER and PASSWORD values blank</li>
    <li>Pass this data to the TD_AITextEmbeddings function</li>
    </ol>
<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Show original data:</b></p>

In [75]:
SELECT TOP 2 * FROM demo_ofs.CFPB_Complaints;

,Date received,Product,Sub-product,Issue,Sub-issue,txt,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,id
1,2024-02-02,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I had a statement generated XXXX XXXX with a due date in XXXX. I paid on XX/XX/XXXX before the bill was due. After the bill was due I received a new statement saying the due date was missed and I now have a past due amount. The statement even shows my payment was made in time. I called mohela and the said the payment went straight to the interest because I paid too early. I pointed out that I paid on time not early. They said the would correct it in 5 to 7 days. I was told at that time not to pay until it was fixed. A week went by and it still showed past due still. I called again and the agent said my payment should have happened because of forbearance and it takes 30 days for XXXX to fix the account. I asked them about my next payment and they said to go ahead and pay. However I mentioned how the manager before said not you pay yet. I was then put on hold to while they confirmed what to do and then they hung up on me while I was on hold. On top of that I keep getting messages about being past due. And my pr,Company believes it acted appropriately as authorized by contract or law,MOHELA,MT,59601,None,Conse,Web,2024-02-02,Closed with explanation,Ye,**,8270781
2,2023-11-12,Credit card,General-purpose credit card or charge card,"Advertising and marketing, including promotional offers",Didn't receive advertised or promotional terms,"I am writing to file a formal complaint against American Express regarding the Delta Gold Card promotion that I participated in. I believe there has been a failure on the part of American Express to fulfill the promised promotional offer. On XXXX, I applied for the American Express Delta Gold Card with the understanding that, upon spending {$2000.00} within the first 6 months, I would be entitled to receive XXXX bonus miles and {$300.00} cash back if I made a XXXX purchase in the first 6 months. I am writing to express my dissatisfaction and frustration as, despite meeting the spending requirement promptly, I have not received the promised bonus miles and cash back. I made purchases, particularly airline tickets through delta, an I have well exceeding the required {$2000.00} within the specified timeframe. Despite my efforts to resolve this matter by contacting American Express customer service on multiple occasions, the issue remains unresolved. I have spoken to numerous agents, and each time I am assure",None,AMERICAN EXPRESS COMPANY,OR,97206,None,Conse,Web,2023-11-12,Closed with monetary relief,Ye,**,7839841


In [76]:
SELECT TOP 2 * FROM demo_ofs.topics_of_interest;

,id,txt
1,2,Identity theft issues at Citibank
2,1,Fradulent activity with Debit Cards at Wells Fargo


<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Configure authorization</b></p>

In [77]:
REPLACE AUTHORIZATION demo_embeddings_auth USER '' PASSWORD '';

Success: 0 rows affected

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Generate Vector Embeddings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the following as input</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Input Data</b>.  Including a partition value to execute the embedding function from a single AMP</li>
    <li><b>Model information</b> including the model name, AWS region of the deployment, and any additional arguments</li>
    <li><b>Authorization</b>.  In this case, the blank passthrough object</li>
    </ul>

In [101]:
SELECT *
FROM AI_TEXTEMBEDDINGS (
    ON (SELECT TOP 2 txt, id, TD_BYONE() p FROM demo_ofs.CFPB_Complaints) AS InputTable
    PARTITION BY p
USING       
     region('us-east-1')
     refreshcredentialtimeseconds('3600')
     Authorization(demo_embeddings_auth)
     apitype('aws')
     modelname('amazon.titan-embed-text-v1')
     --modelargs('{"dimensions":256}')
     textcolumn('txt')
     outputformat('vector')
) as dt;

,txt,id,p,Embedding,Message
1,"I'm disputing some accounts on my credit report due to identity theft. Other bureaus removed them, but yours hasn't. Creditors promised to fix it, so please remove the accounts ASAP.",8266390,28345,...,
2,"It is entirely unacceptable to see late payments noted on my account, and I vehemently request that you promptly correct this mistake. Please take immediate action to rectify my account and guarantee its accurate reflection of my payment history.",8131638,28345,...,


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 2 - Normalize the vector values</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vector normalization is the process of scaling a vector to have a magnitude (length) of 1, while preserving its direction. This resulting vector is called a unit vector. It's essentially dividing each component of the vector by its length.  This makes some calculations much more efficient, including some of the search and indexing operations.  The ClearScape Analytics function <b>TD_VectorNormalize</b> will perform this operation at scale on our VECTOR datatype.</p>

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Execute the VectorNormalize function against the output of the embedding function</b></p>


In [79]:
SELECT * FROM TD_Vectornormalize(
       ON (SELECT txt, id, Embedding, Embedding Embedding_Normalized
            FROM AI_TEXTEMBEDDINGS (
                ON (SELECT TOP 2 txt, id, TD_BYONE() p FROM demo_ofs.CFPB_Complaints) AS InputTable
                PARTITION BY p
            USING       
                 region('us-east-1')
                 refreshcredentialtimeseconds('3600')
                 Authorization(demo_embeddings_auth)
                 apitype('aws')
                 modelname('amazon.titan-embed-text-v1')
                 modelargs('{}')
                 textcolumn('txt')
                 outputformat('vector')
    ) as ve) AS InputTable
USING
    IDColumns('id')
    TargetColumns('Embedding_Normalized')
    Approach('UNITVECTOR')
    Accumulate('txt','Embedding')
    EmbeddingSize(1536)
) AS dt;

,id,Embedding_Normalized,txt,Embedding
1,8266390,...,"I'm disputing some accounts on my credit report due to identity theft. Other bureaus removed them, but yours hasn't. Creditors promised to fix it, so please remove the accounts ASAP.",...
2,8131638,...,"It is entirely unacceptable to see late payments noted on my account, and I vehemently request that you promptly correct this mistake. Please take immediate action to rectify my account and guarantee its accurate reflection of my payment history.",...


<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Create new volatile tables to store the final embeddings and normalized values.</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create new volatile tables passing all the functions in a single expression.  Use Accumulate clause to return original embedding and comments text.</p>

In [103]:
DROP TABLE topics_embeddings_Normalized 

Success: 0 rows affected

In [104]:
CREATE VOLATILE TABLE topics_embeddings_Normalized AS (
    SELECT * FROM TD_Vectornormalize(
           ON (SELECT txt, id, Embedding, Embedding Embedding_Normalized
                FROM AI_TEXTEMBEDDINGS (
                    ON (SELECT txt, id, TD_BYONE() p FROM demo_ofs.topics_of_interest) AS InputTable
                    PARTITION BY p
                USING       
                     region('us-east-1')
                     refreshcredentialtimeseconds('3600')
                     Authorization(demo_embeddings_auth)
                     apitype('aws')
                     modelname('amazon.titan-embed-text-v1')
                     modelargs('{}')
                     textcolumn('txt')
                     outputformat('vector')
        ) as ve) AS InputTable
    USING
        IDColumns('id')
        TargetColumns('Embedding_Normalized')
        Approach('UNITVECTOR')
        Accumulate('txt','Embedding')
        EmbeddingSize(1536)
) AS d) WITH DATA
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [81]:
SELECT TOP 2 * FROM topics_embeddings_Normalized;

,id,Embedding_Normalized,txt,Embedding
1,3,...,Multiple account openings without authorization,...
2,6,...,Cant get money out of ATM,...


In [22]:
DROP TABLE CFPB_embeddings_Normalized

Success: 0 rows affected

In [82]:
CREATE VOLATILE TABLE CFPB_embeddings_Normalized AS (
    SELECT * FROM TD_Vectornormalize(
           ON (SELECT txt, id, Embedding, Embedding Embedding_Normalized
                FROM AI_TEXTEMBEDDINGS (
                    ON (SELECT TOP 1000 txt, id, TD_BYONE() p FROM demo_ofs.CFPB_Complaints) AS InputTable --only pass 1000 rows to the embedding function
                    PARTITION BY p
                USING       
                     region('us-east-1')
                     refreshcredentialtimeseconds('3600')
                     Authorization(demo_embeddings_auth)
                     apitype('aws')
                     modelname('amazon.titan-embed-text-v1')
                     modelargs('{}')
                     textcolumn('txt')
                     outputformat('vector')
        ) as ve) AS InputTable
    USING
        IDColumns('id')
        TargetColumns('Embedding_Normalized')
        Approach('UNITVECTOR')
        Accumulate('txt','Embedding')
        EmbeddingSize(1536)
) AS d) WITH DATA
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [83]:
SELECT TOP 2 * FROM CFPB_embeddings_Normalized;

,id,Embedding_Normalized,txt,Embedding
1,8441547,...,"I got an error when trying to log in which said "" Online Access Temporarily Blocked '' and to call. No big deal, I've been an Ally customer for 10 years and this has happened before. I call and they say my account is under investigation because my personal check from XXXX XXXX XXXX I deposited $ XXXX to Ally bank. Ally flag it and called suspicious when they already withdrew $ XXXX from my BOA. The process will take 10 days to clear the check. Im fine with that but why locked access to my whole Ally bank accounts? Usually when I call, they immediately unlock the account and sometimes make me change the password. But to lock me out of my accounts for 10 days for no reason is wild when I already explained to them what I did. I get my direct deposits through Ally and use it to pay bills, so this is really a frustrating situation.",...
2,7725158,...,"I paid off my Caesars credit card, through comenity bank, in full on XXXX XXXX, 2023. As I was paying the card off online, I noticed that my card was registered to an old email account that I no longer have access to. I successfully changed my email address on their site, not knowing that this would raise a security concern and would suspend my account. On XX/XX/XXXX, I attempted to use the card again for the first time, and it was declined. I was never informed or emailed that my account had been suspended due to a security concern. I called comenity bank first thing Monday morning on XX/XX/XXXX. The individual that I spoke to, had me verify several personal aspects of information including my email and phone number. I informed him that I recently changed my email because I no longer had access to the one that was originally listed on the account. After a short hold, he informed me that Comenity bank was in the process of reviewing my account and that I would receive a letter in a few days regarding the secu",...


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 2 - Perform Vector Distance calculations</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_VectorDistance</a> function will take a table of input Vectors and a table of reference vectors and returns a table that contains the distance between target-reference pairs.  Since this function scans every row and performs the distance calculation, it is resource-intensive and usually suited to a lower number of records.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Additionally, perform a simple join to display the original complaints, topic of interest, and distance calculations</p>

In [84]:
SELECT TOP 10 target_id, reference_id, distancetype, cast(distance as decimal(36,8)) as distance FROM TD_VECTORDISTANCE (
    ON CFPB_Embeddings_Normalized AS TargetTable
    ON topics_embeddings_Normalized AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('Embedding_Normalized')
    RefIDColumn('id')
    RefFeatureColumns('Embedding_Normalized')
    DistanceMeasure('cosine')
    topk(1)
) AS dt order by 3,1,2,4;

,target_id,reference_id,distancetype,distance
1,7664854,1,cosine,.34036222
2,7665970,5,cosine,.54999474
3,7666622,1,cosine,.59587046
4,7666697,1,cosine,.74213127
5,7666918,2,cosine,.63698149
6,7667535,1,cosine,.56216897
7,7668469,2,cosine,.58067852
8,7668998,2,cosine,.71561881
9,7669137,4,cosine,.67951255
10,7671001,4,cosine,.69058583


<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Join the results back to the original complaints and topics</b></p>

In [85]:
SELECT TOP 2 * FROM demo_ofs.CFPB_Complaints;

,Date received,Product,Sub-product,Issue,Sub-issue,txt,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,id
1,2024-02-02,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I had a statement generated XXXX XXXX with a due date in XXXX. I paid on XX/XX/XXXX before the bill was due. After the bill was due I received a new statement saying the due date was missed and I now have a past due amount. The statement even shows my payment was made in time. I called mohela and the said the payment went straight to the interest because I paid too early. I pointed out that I paid on time not early. They said the would correct it in 5 to 7 days. I was told at that time not to pay until it was fixed. A week went by and it still showed past due still. I called again and the agent said my payment should have happened because of forbearance and it takes 30 days for XXXX to fix the account. I asked them about my next payment and they said to go ahead and pay. However I mentioned how the manager before said not you pay yet. I was then put on hold to while they confirmed what to do and then they hung up on me while I was on hold. On top of that I keep getting messages about being past due. And my pr,Company believes it acted appropriately as authorized by contract or law,MOHELA,MT,59601,None,Conse,Web,2024-02-02,Closed with explanation,Ye,**,8270781
2,2023-11-12,Credit card,General-purpose credit card or charge card,"Advertising and marketing, including promotional offers",Didn't receive advertised or promotional terms,"I am writing to file a formal complaint against American Express regarding the Delta Gold Card promotion that I participated in. I believe there has been a failure on the part of American Express to fulfill the promised promotional offer. On XXXX, I applied for the American Express Delta Gold Card with the understanding that, upon spending {$2000.00} within the first 6 months, I would be entitled to receive XXXX bonus miles and {$300.00} cash back if I made a XXXX purchase in the first 6 months. I am writing to express my dissatisfaction and frustration as, despite meeting the spending requirement promptly, I have not received the promised bonus miles and cash back. I made purchases, particularly airline tickets through delta, an I have well exceeding the required {$2000.00} within the specified timeframe. Despite my efforts to resolve this matter by contacting American Express customer service on multiple occasions, the issue remains unresolved. I have spoken to numerous agents, and each time I am assure",None,AMERICAN EXPRESS COMPANY,OR,97206,None,Conse,Web,2023-11-12,Closed with monetary relief,Ye,**,7839841


In [86]:
SELECT TOP 2 * FROM demo_ofs.topics_of_interest;

,id,txt
1,2,Identity theft issues at Citibank
2,1,Fradulent activity with Debit Cards at Wells Fargo


In [87]:
SELECT TOP 10 c.id complaint_id, r.txt topic, c.txt complaint, d.distance

FROM (SELECT target_id, reference_id, distancetype, cast(distance as decimal(36,8)) as distance FROM TD_VECTORDISTANCE (
    ON CFPB_Embeddings_Normalized AS TargetTable
    ON topics_embeddings_Normalized AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('Embedding_Normalized')
    RefIDColumn('id')
    RefFeatureColumns('Embedding_Normalized')
    DistanceMeasure('cosine')
    topk(1)
) AS dt) d
    
JOIN demo_ofs.CFPB_Complaints c
    ON c.id = d.target_id
JOIN demo_ofs.topics_of_interest r
    ON r.id = d.reference_id

ORDER BY d.distance;

,complaint_id,topic,complaint,distance
1,7805942,Fradulent activity with Debit Cards at Wells Fargo,Wells Fargo Bank allowed illegal unauthorized fraudulent transactions and charges and has not accepted responsibility for the illegal unauthorized fraudulent transactions and charges,.26592196
2,8309418,Fradulent activity with Debit Cards at Wells Fargo,"On XX/XX/XXXX I found that Wells Fargo had withdrawn {$400.00} from our savings account ( joint account held by my spouse XXXX XXXX and myself ). When I contacted the bank, they stated that this was an issue with a levy due to a Wells Fargo credit card I had 9 years ago. When complaining about why there was no warning given or documentation of this, they told me that they did not have to provide that information legally. RIDICULOUS! When I spoke with my spouse we discussed the circumstances with the credit card in question and were able to jump to our own conclusions as an explanation of this mess. This is the context : In XXXX I was in the hospital due to a XXXX XXXX from XXXX and subsequent cardiac arrests I underwent in XXXX, XXXX because of a flu virus that I contracted. I was in a coma for a month 's time and have been recovering since that time and I am legally XXXX while receiving Social Security XXXX because of my inability to work due to chronic pulmonary and cardiac conditions. At the time of my h",.27610904
3,8370007,Fradulent activity with Debit Cards at Wells Fargo,"On XX/XX/2024 I attempted to pay from my checking account that I used to regularly make payments, {$23000.00} I was told I could not because it was more than the balance. So I authorized a payment of the full balance due of {$22000.00}. The money was withdrawn from my bank account and paid to Wells Fargo not later than XX/XX/2024. Wells Fargo put a "" hold '' on that payment and denied 2 subsequent purchases to my embarrassment claiming I could not use my credit limit because they had a 14 day "" hold '' on the payment. My bank confirmed to me that Wells Fargo received the payment on XXXX XXXX. I thereupon told Wells Fargo to close my account because I will not deal with a Bank that holds my payment for "" no good reason ''. Something has to be done about actions like this.",.27698797
4,7810386,Fradulent activity with Debit Cards at Wells Fargo,"My Wells Fargo Bank visa credit card was fraudulently used in multiple establishments ( some of which I have never been to ) during the period XX/XX/XXXX - XX/XX/XXXX. I contacted the bank and they reversed the charges. Later, they put the charges back with the following reason. The credit cards were in your possession at the time of fraud ''. Their contention is that the card was used physically with the chip in place. They said I should be responsible for the charges. I am absolutely sure these charges were not incurred by me nor my wife ( the co-holder of the credit card ). It appears that the card was indeed not in our possession based on the banks reasoning. I would like this to be taken up with the bank, so that we are not penalized with over {$4000.00} of fraudulent charges. This is certainly unfair to us! We have been a Wells Fargo customer for over 25 years. Why would we report fraudulent charges if they were indeed incurred by us? Hoping you can assist us in this matter. ( Wells Fargo comp",.28157239
5,7807073,Cant get money out of ATM,"XX/XX/2023 I went to an atm in my banks network, ATM Terminal XXXX at XXXX to withdraw {$400.00} and the atm did not dispense the money. However, it did take the money out of my account. I called my bank and was told to get ahold of the number that was on the atm. I called the XXXX XXXX number located on the machine and was told that I needed to call my bank again to file a dispute. I called my bank once again to file a dispute and a few days later, the dispute was denied because they found no fault and that I had received the money, when I in fact, had not received it. I have since filed 6 rebuttals on the denied disput

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 3 - leverage an HSNW index for fast search</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_HSNW</a> (Hierarchical Navigable Small Worlds) function will create an index table representing....  This index model can then be used by the TD_HSNWPredict function to perform extremely efficient similarity searches.</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Train the model using the Vector Datatype as input</li>
    <li>Predict nearest matches using the topics embeddings as input</li>
    <li>Join the original data for human-readable results</li>
    </ol>
    
<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Train the model</b></p>

In [105]:
DROP TABLE hnsw_model

Success: 0 rows affected

In [106]:
SELECT * FROM TD_HNSW (
    ON CFPB_Embeddings_Normalized AS InputTable
    OUT VOLATILE TABLE OutputTable(hnsw_model)
USING
    IdColumn('id')
    VectorColumn('Embedding_Normalized')
    DistanceMeasure('euclidean')
    EmbeddingSize(1536)
    ApplyHeuristics('true')
) as dt;

Success: 0 rows affected

,message
1,HNSW Graph Construction completed successfully and stored in the OutputTable.


<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Return similar results using topics as input</b></p>

In [107]:
SELECT TOP 10 * FROM
TD_HNSWPREDICT (
    ON hnsw_model AS ModelTable
    ON topics_embeddings_Normalized AS InputTable DIMENSION
    USING
    IdColumn('id')
    VectorColumn('Embedding_Normalized')
    TopK(2)
    OutputNearestVector('true')
) T ORDER BY 1,3;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:9124:Segmentation Violation in AMP: Please do not resubmit the last request.

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Join the results back to the original complaints and topics</b></p>

In [37]:
SELECT TOP 10 c.id complaint_id, r.txt topic, c.txt complaint, d.distance

FROM (SELECT * FROM
TD_HNSWPREDICT (
    ON hnsw_model AS ModelTable
    ON topics_embeddings_Normalized AS InputTable DIMENSION
    USING
    IdColumn('id')
    VectorColumn('Vector_Data')
    EfSearch(16)
    TopK(2)
    OutputNearestVector('true')
) T) d
    
    
JOIN demo_ofs.CFPB_Complaints c
    ON c.id = d.nearest_neighbor_id
JOIN demo_ofs.topics_of_interest r
    ON r.id = d.id

ORDER BY d.distance;

,complaint_id,topic,complaint,distance
1,8297421,Identity theft issues at Citibank,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,0.7541622358478657
2,8297421,Fradulent activity with Debit Cards at Wells Fargo,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,0.8265277541568549
3,7907705,Identity theft issues at Citibank,I received a capital one card in the mail on XX/XX/XXXX I never requested nor do I even have an account with them after they made an inquiry on my experian account on XX/XX/XXXX. I never requested this inquiry either so they opened a credit card without my consent and brought my credit score down.,0.8282252455116907
4,7720134,App issues when transacting with bank,I have a credit card with goldman sachs/apple . I have had difficulties paying the card since having only XXXX payments. I requested them to work with my debt management company and they refused. I have called several times and also sent a letter to them asking if they would reduce interest and setup a payment plan with me directly. They have refused to work with me in any way and continue to only make the payment higher and more impossible for me to catch up on and have made the regular monthly payment besides the back payment unfairly high. I called today XX/XX/18 and asked if there was any way I could resume monthly payments as they were and they said that the regular monthly payment besides the back monthly payment was now more than $ XXXX which previously was $ XXXX. So even if I paid the back payments I would now have to pay a $ XXXX monthly payment which they know is impossible for me to pay and also seems to be abusive of the collections system. I have tried very hard to do everything in my power to w,0.8314917937167623
5,7720134,Fradulent activity with Debit Cards at Wells Fargo,I have a credit card with goldman sachs/apple . I have had difficulties paying the card since having only XXXX payments. I requested them to work with my debt management company and they refused. I have called several times and also sent a letter to them asking if they would reduce interest and setup a payment plan with me directly. They have refused to work with me in any way and continue to only make the payment higher and more impossible for me to catch up on and have made the regular monthly payment besides the back payment unfairly high. I called today XX/XX/18 and asked if there was any way I could resume monthly payments as they were and they said that the regular monthly payment besides the back monthly payment was now more than $ XXXX which previously was $ XXXX. So even if I paid the back payments I would now have to pay a $ XXXX monthly payment which they know is impossible for me to pay and also seems to be abusive of the collections system. I have tried very hard to do everything in my power to w,0.8372770119410285
6,7756774,App issues when transacting with bank,"I received an email from FNBO-Direct on XX/XX/ confirming they received my application for an account. I was given an application number. I had not opened or requested an account and thought it was a fraudulent email from a fake bank. I confirmed that FNBO-Direct is the First National Bank of Omaha-Direct and I that are an actual bank. I forwarded 

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use TD_HNSWSummary to create a human-readable model output</b></p>

In [108]:
SELECT amp_id, graph_id, node_id, layer_id, input_row_id, cast(node_vector
as varchar(60)) as node_vector, num_neighbors, cast(neighbor_node_id as
varchar(60)) as neighbor_node_id, cast(model_info as varchar(500)) as model_info 

FROM TD_HNSWSummary(
    ON hnsw_model as ModelTable
) as dt
ORDER by 1,9

,amp_id,graph_id,node_id,layer_id,input_row_id,node_vector,num_neighbors,neighbor_node_id,model_info
1,0,0,3,1,8294256,"-0.231445,-0.640625,-0.0771484,-0.433594,-0.125,0.339844,-0.",10,"1,2,4,5,11,13,15,17,48,70",
2,0,0,5,1,8092334,"-0.154297,-0.220703,-0.263672,-0.148438,-0.0566406,0.205078,",30,"4,1,3,2,6,7,8,11,13,14,17,19,20,22,27,29,30,35,42,45,46,48,5",
3,0,0,6,1,8381471,"-0.136719,-0.527344,-0.192383,-0.048584,-0.0218506,0.396484,",7,"5,9,24,33,40,48,77",
4,0,0,7,1,8578783,"0.0559082,-0.019165,-0.453125,-0.246094,-0.378906,0.25,-0.17",11,"1,5,11,34,49,62,66,72,84,87,95",
5,0,0,9,1,8016516,"-0.335938,-0.137695,-0.306641,0.324219,-0.578125,0.251953,-0",22,"6,8,13,20,28,31,32,37,38,40,42,43,48,53,56,68,73,76,77,82,86",
6,0,0,10,1,8032666,"-0.163086,0.144531,-0.230469,0.0617676,-0.108887,0.206055,-0",12,"8,17,19,44,46,57,67,70,85,87,105,115",
7,0,0,11,1,7729234,"-0.0922852,-0.124512,-0.0078125,-0.10791,0.0986328,0.0554199",16,"8,5,7,3,12,18,23,25,29,54,64,67,76,79,92,118",
8,0,0,12,1,8139152,"0.246094,-0.0515137,0.0419922,-0.34375,-0.243164,0.0449219,0",6,"11,21,25,38,72,78",
9,0,0,13,1,8308099,"-0.138672,-0.155273,-0.332031,0.0078125,-0.00836182,0.349609",15,"5,9,3,17,24,26,27,29,36,46,55,56,73,105,125",
10,0,0,14,1,8295823,"0.151367,-0.152344,-0.194336,0.0717773,0.223633,0.15332,-0.0",12,"5,15,16,21,24,38,47,48,61,69,89,118",


In [69]:
%disconnect vs_demo

Success: 'vs_demo' disconnected